In [34]:
import numpy as np
import pandas as pd
import networkx as nx



import os
import sys
import inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0, parentdir)

sys.path.insert(0, '..')

import Utils as Utils

import scanpy as sc

In [36]:
data_path = "scRNA-seq-Gene expression profile at single cell level of a rare seen and young tracheal adenoid cystic carcinoma/";
file_name = "GSE245170_RAW/Tumor";

adata = sc.read_10x_mtx(data_path+file_name);
adata


AnnData object with n_obs × n_vars = 16325 × 36601
    var: 'gene_ids', 'feature_types'

In [38]:
adata.var["mito"] = adata.var_names.str.startswith("MT-");
sc.pp.filter_cells(adata, min_genes=3000);
sc.pp.filter_genes(adata, min_cells=500);
adata

AnnData object with n_obs × n_vars = 1728 × 4980
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells'

In [40]:
# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)


In [42]:
sc.pp.highly_variable_genes(adata, n_top_genes=300, inplace=True);
adata


AnnData object with n_obs × n_vars = 1728 × 4980
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [44]:
adata.var.highly_variable
adata.X
print(adata)
print(adata.X)
display(adata.obs)
display(adata.var)

AnnData object with n_obs × n_vars = 1728 × 4980
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
<Compressed Sparse Column sparse matrix of dtype 'float32'
	with 4685255 stored elements and shape (1728, 4980)>
  Coords	Values
  (3, 0)	0.6229950785636902
  (5, 0)	0.9672199487686157
  (12, 0)	0.6182351112365723
  (13, 0)	0.33898186683654785
  (17, 0)	0.6395485997200012
  (18, 0)	0.696438729763031
  (20, 0)	0.8558768033981323
  (22, 0)	0.9062440395355225
  (25, 0)	0.7986065745353699
  (27, 0)	0.48109710216522217
  (30, 0)	0.4648277759552002
  (34, 0)	0.7259008884429932
  (35, 0)	0.6967246532440186
  (39, 0)	0.3021407127380371
  (42, 0)	0.5877075791358948
  (45, 0)	0.9287208914756775
  (47, 0)	1.061889410018921
  (49, 0)	0.7308778166770935
  (51, 0)	0.2629909813404083
  (54, 0)	1.1620252132415771
  (59, 0)	1.3011486530303955
  (61, 0)	1.0878385305404663
  (62, 0)	1.594151854515

,n_genes
AAACCCAAGAAGCCAC-1,3263
AAACGAAGTAACTGCT-1,3735
AAACGAAGTTGTCTAG-1,3886
AAACGCTGTGATACCT-1,3281
AAAGGTAAGATTGAGT-1,3814
...,...
TTTGATCCATCATGAC-1,4961
TTTGGAGTCAAAGAAC-1,3199
TTTGGTTAGGTCACAG-1,3036
TTTGTTGGTACCCGCA-1,3658


,gene_ids,feature_types,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
NOC2L,ENSG00000188976,Gene Expression,False,715,False,0.416492,0.107322,-0.598550
HES4,ENSG00000188290,Gene Expression,False,1387,False,1.647143,1.456193,0.230056
ISG15,ENSG00000187608,Gene Expression,False,653,True,0.653402,2.072292,3.440248
SDF4,ENSG00000078808,Gene Expression,False,1050,False,0.695128,0.347632,-0.259990
UBE2J2,ENSG00000160087,Gene Expression,False,660,False,0.374101,0.055129,-0.830181
...,...,...,...,...,...,...,...,...
MT-ND4L,ENSG00000212907,Gene Expression,True,1167,False,0.787141,0.315566,-0.328788
MT-ND4,ENSG00000198886,Gene Expression,True,1728,False,4.291566,2.582808,0.105474
MT-ND5,ENSG00000198786,Gene Expression,True,1727,False,2.712791,1.447904,-0.517526
MT-ND6,ENSG00000198695,Gene Expression,True,621,False,0.372414,0.186308,-0.248017


In [46]:
file_name = "filtereddata";

adata.write_csvs(data_path+file_name, skip_data=False, sep=',')

In [47]:
obs = pd.read_csv(data_path+file_name+"/obs.csv",index_col=0);
display(obs);
var = pd.read_csv(data_path+file_name+"/var.csv");
display(var);

dataset = pd.read_csv(data_path+file_name+"/X.csv",names=var['gene_ids']);
dataset.index = obs.index;
display(dataset);

highly_variable_genes = var[var['highly_variable']]['gene_ids'].values;
dataset = dataset[highly_variable_genes];
display(dataset);

,n_genes
AAACCCAAGAAGCCAC-1,3263
AAACGAAGTAACTGCT-1,3735
AAACGAAGTTGTCTAG-1,3886
AAACGCTGTGATACCT-1,3281
AAAGGTAAGATTGAGT-1,3814
...,...
TTTGATCCATCATGAC-1,4961
TTTGGAGTCAAAGAAC-1,3199
TTTGGTTAGGTCACAG-1,3036
TTTGTTGGTACCCGCA-1,3658


,Unnamed: 0,gene_ids,feature_types,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
0,NOC2L,ENSG00000188976,Gene Expression,False,715,False,0.416492,0.107322,-0.598550
1,HES4,ENSG00000188290,Gene Expression,False,1387,False,1.647143,1.456193,0.230056
2,ISG15,ENSG00000187608,Gene Expression,False,653,True,0.653402,2.072292,3.440248
3,SDF4,ENSG00000078808,Gene Expression,False,1050,False,0.695128,0.347632,-0.259990
4,UBE2J2,ENSG00000160087,Gene Expression,False,660,False,0.374101,0.055129,-0.830181
...,...,...,...,...,...,...,...,...,...
4975,MT-ND4L,ENSG00000212907,Gene Expression,True,1167,False,0.787141,0.315566,-0.328788
4976,MT-ND4,ENSG00000198886,Gene Expression,True,1728,False,4.291566,2.582808,0.105474
4977,MT-ND5,ENSG00000198786,Gene Expression,True,1727,False,2.712791,1.447904,-0.517526
4978,MT-ND6,ENSG00000198695,Gene Expression,True,621,False,0.372414,0.186308,-0.248017


,ENSG00000188976,ENSG00000188290,ENSG00000187608,ENSG00000078808,ENSG00000160087,ENSG00000127054,ENSG00000175756,ENSG00000221978,ENSG00000242485,ENSG00000179403,...,ENSG00000198804,ENSG00000198712,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727
AAACCCAAGAAGCCAC-1,0.000000,2.073649,0.000000,0.000000,0.000000,0.769629,0.000000,0.000000,0.769629,0.000000,...,5.580650,5.311530,5.526792,5.535973,3.973183,0.769629,4.623304,3.084721,0.769629,5.109413
AAACGAAGTAACTGCT-1,0.000000,1.123479,0.000000,1.414226,0.711855,0.000000,1.123479,1.123479,0.711855,0.000000,...,4.709560,4.367326,5.154848,4.700168,2.925420,0.711855,4.179629,2.868142,0.000000,4.865094
AAACGAAGTTGTCTAG-1,0.000000,0.449824,0.000000,1.604677,1.185445,0.000000,1.810307,0.000000,1.483470,0.000000,...,5.347284,5.020780,5.247844,4.966871,2.923651,0.000000,4.642605,2.793430,1.185445,4.970820
AAACGCTGTGATACCT-1,0.622995,1.279130,0.622995,1.003938,0.000000,0.622995,1.494704,0.000000,1.003938,0.000000,...,3.227086,3.469546,3.595795,4.250783,2.696791,0.622995,2.696791,1.494704,1.003938,3.118507
AAAGGTAAGATTGAGT-1,0.000000,2.681388,0.000000,0.920962,0.000000,0.562955,1.392130,0.562955,0.920962,0.000000,...,5.758733,5.297264,5.666358,5.345274,3.987311,0.562955,4.746095,2.146929,0.000000,5.218583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGATCCATCATGAC-1,0.494864,1.270070,1.071867,1.435412,0.494864,0.824417,1.071867,0.000000,1.270070,2.670403,...,4.704899,4.354566,4.669510,4.492651,2.795243,0.000000,3.905219,2.625068,0.494864,4.639031
TTTGGAGTCAAAGAAC-1,0.000000,2.638774,0.000000,0.000000,0.893637,0.000000,1.357897,1.357897,1.357897,0.000000,...,5.083169,4.885860,5.240267,4.989373,3.573920,0.893637,4.387265,2.736964,0.893637,4.745595
TTTGGTTAGGTCACAG-1,0.000000,0.000000,0.000000,0.967956,0.000000,0.967956,0.967956,0.000000,1.450470,0.000000,...,5.524582,4.951593,5.370011,5.007717,3.693404,0.967956,4.526379,3.172028,0.000000,5.274406
TTTGTTGGTACCCGCA-1,0.000000,1.707008,0.643163,0.643163,0.643163,0.000000,1.031396,0.000000,1.528218,0.000000,...,5.560737,5.446898,5.627852,5.650317,4.118496,1.031396,4.694084,3.233457,0.000000,5.411265


,ENSG00000187608,ENSG00000179403,ENSG00000162413,ENSG00000142627,ENSG00000117122,ENSG00000158747,ENSG00000142798,ENSG00000126709,ENSG00000142910,ENSG00000131238,...,ENSG00000211677,ENSG00000100234,ENSG00000100360,ENSG00000077942,ENSG00000177989,ENSG00000102265,ENSG00000102109,ENSG00000157514,ENSG00000029993,ENSG00000147400
AAACCCAAGAAGCCAC-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.199341,0.000000,1.199341,0.000000,0.000000,1.199341,0.000000,0.769629,0.000000,0.000000
AAACGAAGTAACTGCT-1,0.000000,0.000000,0.000000,0.000000,0.711855,1.123479,0.711855,1.639204,0.000000,1.414226,...,0.711855,3.331560,0.000000,4.163619,0.000000,4.942077,0.000000,1.414226,0.000000,0.000000
AAACGAAGTTGTCTAG-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.758968,0.449824
AAACGCTGTGATACCT-1,0.622995,0.000000,0.000000,2.068890,0.000000,0.622995,0.000000,0.000000,1.494704,0.000000,...,0.000000,0.622995,0.622995,1.003938,0.000000,3.864579,0.000000,0.000000,0.000000,0.000000
AAAGGTAAGATTGAGT-1,0.000000,0.000000,0.920962,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.562955,1.564287,0.562955,0.000000,1.184043,0.920962,1.184043,0.000000,0.562955,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGATCCATCATGAC-1,1.071867,2.670403,0.494864,0.494864,0.000000,0.824417,2.833618,0.494864,1.071867,0.000000,...,0.000000,2.232547,0.000000,0.000000,1.071867,2.001852,0.000000,1.071867,0.000000,0.000000
TTTGGAGTCAAAGAAC-1,0.000000,0.000000,0.000000,0.000000,0.893637,0.000000,0.000000,0.000000,0.893637,0.893637,...,0.000000,0.000000,0.893637,0.000000,0.893637,0.893637,0.000000,0.000000,0.893637,0.000000
TTTGGTTAGGTCACAG-1,0.000000,0.000000,0.967956,0.000000,0.000000,1.774558,0.000000,1.450470,0.000000,0.000000,...,0.000000,3.878346,0.967956,4.034365,0.000000,4.034365,0.000000,2.379116,0.000000,0.967956
TTTGTTGGTACCCGCA-1,0.643163,0.000000,0.643163,0.000000,0.643163,1.031396,0.000000,0.000000,0.000000,0.000000,...,1.031396,2.305071,0.000000,0.643163,0.643163,0.643163,0.643163,0.643163,0.000000,0.000000


In [50]:
dataset.to_csv(data_path+file_name+"/filtered_gene_exp_mat.csv",index=None);